# Text analytics

## Datensammlung

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pyautogui  # Automatische Bewegung der Maus
from selenium.webdriver.common.keys import Keys  # ermöglicht Tastatureingabe mit zu nutzen
from selenium.webdriver.common.action_chains import ActionChains  # Tab handling

import pandas as pd
import time
import datetime
import random
import re

#  In Terminal oder Anaconda Prompt:
#  pip install selenium, yfinance, pyautogui

### Datensammlung: Artikel

In [24]:
print(pyautogui.position())

Point(x=823, y=359)


In [31]:
driver = webdriver.Firefox()
driver.maximize_window()

driver.get(f'https://www.onvista.de/news/2023/09-22-markt-weiter-angeschlagen-index-rettet-sich-ueber-200-tagelinie-19-26179188')

time.sleep(random.uniform(3,6))

try:
    pyautogui.moveTo(823,359)  # Bewegt Maus zur angegebene Koordiate. Workaround für Cookie-Banner
    pyautogui.click()

except:
    pass

#table = driver.find_elements(By.TAG_NAME,'table')

#p = driver.find_elements(By.CSS_SELECTOR, 'p.paragraph')
p = driver.find_elements(By.TAG_NAME,'p')


#paragraph = table.find_elements(By.TAG_NAME,'p')
#table[0]
#paragraphs = table.find_elements(By.TAG_NAME('p'))
#table_contents = [p.text for p in paragraphs]

#print(table[0].text)

for i in range(0, len(p)):
    print(p[i].text)
print("---")
print(p[0].text)
print("---")
print(len(p))
print("---")
print(p[len(p)-6].text)
print(p[len(p)-5].text)
print(p[len(p)-4].text)
print(p[len(p)-3].text)
print(p[len(p)-2].text)
print(p[len(p)-1].text)

driver.quit()

Der Dax startete mit einem Abwärtsgap von knapp 80 Punkten in den Freitagshandel. Anschließend fand der Leitindex im Bereich des Augusttiefs eine Unterstüzung und erholte sich somit um die 15.500er Marke.
In der heutigen Dax-Chartanalyse wurde der Bereich bei 15.570 Punkten als prozyklisches bullisches Signal festgelegt.
Dieser Bereich wurde im Laufe des Tages jeweils gegen 14 Uhr und 16:40 knapp erreicht. Diese Marke hielt aber nicht lange genug, um ein nachhaltiges Kaufsignal zu erzeugen. 
Eine Stabilisierung der US-Börsen hat dem Dax heute einen recht versöhnlichen Wochenabschluss beschert. Der Dax ging mit einem Verlust von lediglich 0,09 Prozent bei 15.557,29 Punkten aus dem Handel. 
Die Aussicht auf höhere Zinsen für längere Zeit hatte den Anlegern im Verlauf der Woche die Stimmung verhagelt. Auf Wochensicht verlor der Dax 2,1 Prozent, hielt sich letztlich aber knapp oberhalb der gleitenden 200-Tage-Linie, die als wichtiger Indikator für den langfristigen Trend gilt. Der Kampf zw

In [32]:
driver = webdriver.Firefox()
driver.maximize_window()

dax_companies = {82902 : 'Siemens',
          82849 : 'SAP',
          }


#article.col__content > div:nth-child(4) > table:nth-child(1)

article_data = []

filtered_words = ["Top Aktien", "Top Märkte", "Ratgeber", "onvista media GmbH", "Rechtliche Hinweise", "App", "Social Media"]

for key in dax_companies.keys():  # Iteration über alle keys der Unternehmen aus dem dem dictionary 'dax_companies'
    
    for n in range(0,9):  # Iteration je Unternehmen über 10 Seitenzahlen

        driver.get(f'https://www.onvista.de/news/finder?page={n}&entityType=STOCK&entityValue={key}')  # gefilterte Seiten lassen sich über unternehmensspezifischen key aufrufen
        
        time.sleep(random.uniform(3,6))

        try:
            pyautogui.moveTo(823,359)  # Bewegt Maus zur angegebene Koordiate. Workaround für Cookie-Banner
            pyautogui.click()

        except:
            pass

        articles = driver.find_elements(By.XPATH,'//strong[@class="ov-display--block ov-word-wrap"]')  # Beinhaltet alle Artikel der aktuellen Seite

        for article in articles:

            article.location_once_scrolled_into_view  # Automatisches Scrollen zu jedem Artikel
            time.sleep(random.uniform(2, 4))

            actions = ActionChains(driver)

            actions.key_down(Keys.CONTROL).click(article).key_up(Keys.CONTROL).perform()  # Öffnet Artikel in einem neuen Tab

            driver.switch_to.window(driver.window_handles[1])  # Fokus des Webdriver wird auf neuen Tab gerichtet

            time.sleep(random.uniform(6.2, 13.6))

            try:
                ad = driver.find_element(By.XPATH, '//div[@class="flex-layout flex-layout__align-items--center flex-layout__justify-content--end text-size--xsmall ov-subline"]').text.strip()  # Extraktion dem Titel aus der HTML-Struktur, teweilweise problematisch zu erkennen
                ad_text = ad.split('\n')

                print(ad_text[0])

                if "Werbung" in ad_text[0]:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])  # Fokus zurück zum ersten Tab
                    continue
                
                ad_yt = driver.find_element(By.XPATH, '//h1[@class="SGW9xe"]').text.strip()  # XPATH zu Cookie-Banner von Youtube
                ad_yt_text = ad.split('\n')

                print(ad_yt_text)

                if "YouTube" in ad_yt_text[0]:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue

            except:
                pass
            
            try:   
                article_title = driver.find_element(By.XPATH, '//h1[@class="headline headline--h1 headline--full-width outer-spacing--xxsmall-bottom"]').text.strip()  # Extraktion des Titels aus der HTML-Struktur
                title = article_title.split('\n')
            except:
                title = "NO_TITLE"

            # time_elements = driver.find_elements(By.TAG_NAME, 'time')
            # time_list = []

            # for i, time_element in enumerate(time_elements):
                
            #     if i >= 2:
            #         break

            #     time_text = time_element.text
            #     time_list.append(time_text)

            try:
                article_date = driver.find_element(By.XPATH, '//time[@class="color--cd-anthracite"]').text.strip()  # Extraktion des Datums und Uhrzeit der Veröffentlichung
                date = article_date.split('\n')

            except:
                date = "NO_DATE"


            try:
                #article_text = driver.find_elements(By.CSS_SELECTOR, 'p.paragraph')  # Enthält alle Paragraphen des aktuellen Artikels. Inhalt des Artikel ist am besten über die Paragraphen ansteuerbar.
                article_text = driver.find_elements(By.TAG_NAME,'p')
                time.sleep(random.uniform(1.2, 2))

                text_list = []

                for paragraph in article_text:  # Iteration über alle enthaltenen Paragraphen
                    paragraph.location_once_scrolled_into_view
                    time.sleep(0.5)
                    text_list.append(paragraph.text)  
                    
                    for element in text_list[-7:]:  # Die letzten 7 Elemente sind meist/immer die in 'filtered_words' vorkommende Wörter, welche laut HTML-Struktur leider als paragraph deklariert wurden.

                        if element in filtered_words:
                            text_list.remove(element) 

                    text = '\n'.join(text_list)  # Aneinanderfügung der Paragraphen zu einem String, welcher durch Zeilenumbrüche (\n) getrennet wird.

            except:
                text = "NO_TEXT"



            data = {'Unternehmen' : dax_companies[key], 'Titel' : title[0], 'Date' : date[0], 'Text' : text}  # Zusammenführung der gesammelten Daten des Artikels

            article_data.append(data)

            print(dax_companies[key])
            print(title[0])
            print(date[0])
            print(text)

            driver.close()  # Tab schließen
            driver.switch_to.window(driver.window_handles[0])  # Fokus zurück zum ersten Tab mit der Übersicht der Artikel

        # time.sleep(random.uniform(10, 15))
        # next_page = driver.find_element(By.XPATH,'//div[@class="pagination__button pagination__button--right"]') 
        # next_page.location_once_scrolled_into_view
        # time.sleep(random.uniform(2, 4))
        # next_page.click()

driver.close()

now = datetime.datetime.now()
date_time_str = now.strftime('%Y%m%d_%H%M%S')
date_time_str

df = pd.DataFrame(article_data)

df.to_excel(f'data/Onvista_Articles-{date_time_str}.xlsx', index=False)
df.to_csv(f'data/Onvista_Articles-{date_time_str}.csv', index=False)

Siemens
Markt weiter angeschlagen - Index rettet sich über 200 Tagelinie
22.09.2023, 18:26
Der Dax startete mit einem Abwärtsgap von knapp 80 Punkten in den Freitagshandel. Anschließend fand der Leitindex im Bereich des Augusttiefs eine Unterstüzung und erholte sich somit um die 15.500er Marke.
In der heutigen Dax-Chartanalyse wurde der Bereich bei 15.570 Punkten als prozyklisches bullisches Signal festgelegt.
Dieser Bereich wurde im Laufe des Tages jeweils gegen 14 Uhr und 16:40 knapp erreicht. Diese Marke hielt aber nicht lange genug, um ein nachhaltiges Kaufsignal zu erzeugen. 
Eine Stabilisierung der US-Börsen hat dem Dax heute einen recht versöhnlichen Wochenabschluss beschert. Der Dax ging mit einem Verlust von lediglich 0,09 Prozent bei 15.557,29 Punkten aus dem Handel. 
Die Aussicht auf höhere Zinsen für längere Zeit hatte den Anlegern im Verlauf der Woche die Stimmung verhagelt. Auf Wochensicht verlor der Dax 2,1 Prozent, hielt sich letztlich aber knapp oberhalb der gleitenden

KeyboardInterrupt: 

In [69]:
df = pd.read_csv('Onvista_Articles.csv')
len(df)

175

In [68]:
df[:10]

,Unnamed: 0,Unternehmen,Titel,Date,Text
0,0,Siemens,Insider - Huawei drängt zurück in den Markt fü...,"['20.09.2023, 11:34']",Peking/Shanghai (Reuters) - Der umstrittene ch...
1,1,Siemens,EQS-CMS: Siemens Aktiengesellschaft: Veröffent...,"['18.09.2023, 11:47']",NaN
2,2,Siemens,Aktien Frankfurt: Dax gestärkt durch EZB - Am ...,"['15.09.2023, 15:11']",FRANKFURT (dpa-AFX) - Mit frischem Wind durch ...
3,3,Siemens,Aktien Frankfurt: Dax dreht nach Leitzinsentsc...,"['14.09.2023, 15:12']",FRANKFURT (dpa-AFX) - Am deutschen Aktienmarkt...
4,4,Siemens,ROUNDUP: Bezahlte Freistellung für Väter nach ...,"['14.09.2023, 11:07']",WALLDORF (dpa-AFX) - Der Softwarekonzern SAP i...
5,5,Siemens,ROUNDUP/Aktien Europa Schluss: Leichte Verlust...,"['13.09.2023, 19:10']",PARIS/LONDON (dpa-AFX) - Europas Börsen haben ...
6,6,Siemens,ANALYSE-FLASH: JPMorgan belässt Siemens auf 'O...,"['13.09.2023, 14:05']",NEW YORK (dpa-AFX Broker) - Die US-Bank JPMorg...
7,7,Siemens,Aktien Frankfurt: Leichte Kursgewinne - Anlege...,"['11.09.2023, 15:19']",FRANKFURT (dpa-AFX) - Die von der EZB-Leitzins...
8,8,Siemens,Tesla | Siemens | Kommt der Covestro-Verkauf n...,"['11.09.2023, 12:53']",An dieser Stelle befindet sich externer Inhalt...
9,9,Siemens,Aktien Frankfurt: Dax hält Kursgewinne - Defen...,"['11.09.2023, 12:13']",FRANKFURT (dpa-AFX) - Der deutsche Aktienmarkt...


### Datensammlung: Historische Börsenkurse

In [119]:
import yfinance as yf
import sqlite3

In [108]:
msft = yf.Ticker("MSFT")

tickers = ["SPY", "AAPL", "MSFT"]
# get all stock info
#msft.info

In [122]:
data = yf.download('AAPL')

DEBUG    Entering download()
DEBUG     Disabling multithreading because DEBUG logging enabled
DEBUG     Entering history()
ERROR     
          1 Failed download:
ERROR     ['AAPL']: OperationalError('unable to open database file')
DEBUG     ['AAPL']: Traceback (most recent call last):
            File "c:\Users\marin\anaconda3\lib\site-packages\peewee.py", line 3192, in connect
              self._state.set_connection(self._connect())
            File "c:\Users\marin\anaconda3\lib\site-packages\peewee.py", line 3536, in _connect
              conn = sqlite3.connect(self.database, timeout=self._timeout,
          sqlite3.OperationalError: unable to open database file
          
          During handling of the above exception, another exception occurred:
          
          Traceback (most recent call last):
            File "c:\Users\marin\anaconda3\lib\site-packages\yfinance\multi.py", line 275, in _download_one
              data = Ticker(ticker, session=session).history(
         

In [123]:

data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [103]:
hist = msft.history(period="1mo")

DEBUG    Entering history()


OperationalError: unable to open database file